In [2]:
from pathlib import Path
from datetime import datetime, timedelta
import time
import sys
import os
import requests
import pickle
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler

In [3]:
# Intialising Alpaca API
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    'https://paper-api.alpaca.markets/',
    api_version = "v2"
)

In [4]:
# User inputs
crypto = 'BTCUSD'
investment = 1000

# Variables
price = (api.get_latest_crypto_bar(crypto, 'CBSE')).c
amount = investment/price
model_file_path = Path('../Resources/model.pkl')
scaler_file_path = Path('../Resources/scaler.pkl')

In [5]:
# Load the model
model = pickle.load(open('/Users/michael1/Documents/FinTech/Project-2/Resources/model.pkl','rb'))

In [6]:
# Prepare the data
start_date = (datetime.today().date() - timedelta(days=10)).isoformat()
data = api.get_crypto_bars(
    symbol=crypto,
    timeframe='1H',
    start=start_date
).df

data = data.loc[data['exchange'] == 'CBSE']
data['sma_fast'] = data['close'].rolling(window=3).mean()
data['sma_slow'] = data['close'].rolling(window=10).mean()
data = data.dropna()

X = data[['sma_fast', 'sma_slow']]

In [7]:
# Scale X
scaler = pickle.load(open(scaler_file_path, 'rb'))
X_scaled = scaler.transform(X)

In [8]:
# Predict the y variable
signal = (model.predict(X_scaled))
entry_exit = (signal[1] - signal[0])/2

In [48]:
# Initiate buying/selling
if entry_exit == 1:
    api.submit_order(
        symbol=crypto,
        qty=amount,
        time_in_force='gtc'
    )
    print(f'Successfully bought {round(amount, 5)} of {crypto}.')
elif entry_exit == -1:
    api.submit_order(
        symbol=crypto,
        qty=amount,
        side='sell',
        time_in_force = 'gtc'
    )
    print(f'Successfully sold {round(amount, 5)} of {crypto}.')
else:
    print(f'Waiting/holding...')


Waiting/holding...


In [ ]:
# Adding entry/exit